In [ ]:
# Cell 1: Setup Environment (Uninstall existing PyTorch and install specified versions)
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --extra-index-url https://download.pytorch.org/whl/cu118


Found existing installation: torch 2.0.1+cu118
Uninstalling torch-2.0.1+cu118:
  Successfully uninstalled torch-2.0.1+cu118
Found existing installation: torchvision 0.15.2+cu118
Uninstalling torchvision-0.15.2+cu118:
  Successfully uninstalled torchvision-0.15.2+cu118
Found existing installation: torchaudio 2.0.2+cu118
Uninstalling torchaudio-2.0.2+cu118:
  Successfully uninstalled torchaudio-2.0.2+cu118
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp311-cp311-linux_x86_64.whl (2267.3 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.15.2%2Bcu118-cp311-cp311-linux_x86_64.whl (6.1 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.0.2%2Bcu118-cp311-cp311-linux_x86_64.whl (4.4 MB)


In [ ]:
# Cell X: Downgrade NumPy to a version below 2.0 (e.g., 1.24.3)
!pip install numpy==1.24.3


In [ ]:
# Cell 2: Mount Google Drive and Download CamVid Dataset from Kaggle

from google.colab import drive
drive.mount('/content/drive')

# Create a directory on Drive to store CamVid
!mkdir -p /content/drive/MyDrive/CamVid

# Upload your Kaggle API key (kaggle.json) when prompted
from google.colab import files
uploaded = files.upload()  # Upload kaggle.json

# Set up Kaggle credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the CamVid dataset from Kaggle (dataset: "carlolepelaars/camvid")
!kaggle datasets download -d carlolepelaars/camvid

# Unzip the downloaded dataset to /content/camvid
!unzip camvid.zip -d /content/camvid

# Copy the unzipped CamVid folder to Google Drive for persistence
!cp -r /content/camvid/CamVid /content/drive/MyDrive/CamVid

%env CUDA_LAUNCH_BLOCKING=1


Mounted at /content/drive


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/carlolepelaars/camvid
License(s): CC-BY-NC-SA-4.0
 92% 528M/575M [00:00<00:00, 1.32GB/s]
100% 575M/575M [00:00<00:00, 1.33GB/s]
Archive:  camvid.zip
  inflating: /content/camvid/CamVid/class_dict.csv  
  inflating: /content/camvid/CamVid/test/0001TP_006690.png  
  inflating: /content/camvid/CamVid/test/0001TP_006720.png  
  inflating: /content/camvid/CamVid/test/0001TP_006750.png  
  inflating: /content/camvid/CamVid/test/0001TP_006780.png  
  inflating: /content/camvid/CamVid/test/0001TP_006810.png  
  inflating: /content/camvid/CamVid/test/0001TP_006840.png  
  inflating: /content/camvid/CamVid/test/0001TP_006870.png  
  inflating: /content/camvid/CamVid/test/0001TP_006900.png  
  inflating: /content/camvid/CamVid/test/0001TP_006930.png  
  inflating: /content/camvid/CamVid/test/0001TP_006960.png  
  inflating: /content/camvid/CamVid/test/0001TP_006990.png  
  inflating: /content/camvid/CamVid/test/0001TP_

In [ ]:
    # Cell 3: Set Up Dataset Paths and Verify Folder Structure

    import os

    # Update dataset_dir to include the extra folder level
    dataset_dir = "/content/drive/MyDrive/CamVid/CamVid"

    # Define directories for images and masks based on the CamVid structure
    train_image_dir = os.path.join(dataset_dir, "train")
    train_mask_dir  = os.path.join(dataset_dir, "train_labels")
    val_image_dir   = os.path.join(dataset_dir, "val")
    val_mask_dir    = os.path.join(dataset_dir, "val_labels")
    test_image_dir  = os.path.join(dataset_dir, "test")
    test_mask_dir   = os.path.join(dataset_dir, "test_labels")

    # Verify the folder structure by printing the number of files in each directory
    print("Number of training images:", len(os.listdir(train_image_dir)))
    print("Number of training masks:", len(os.listdir(train_mask_dir)))
    print("Number of validation images:", len(os.listdir(val_image_dir)))
    print("Number of validation masks:", len(os.listdir(val_mask_dir)))
    print("Number of test images:", len(os.listdir(test_image_dir)))
    print("Number of test masks:", len(os.listdir(test_mask_dir)))


Number of training images: 369
Number of training masks: 369
Number of validation images: 100
Number of validation masks: 100
Number of test images: 232
Number of test masks: 232


In [ ]:
# Cell 4: Pair Image Files with Mask Files

def get_pairs(image_dir, mask_dir):
    """
    Returns a list of tuples (image_path, mask_path) for images with corresponding mask files.
    """
    image_files = sorted(os.listdir(image_dir))
    pairs = []
    for f in image_files:
        base = os.path.splitext(f)[0]
        mask_file = base + "_L.png"
        mask_path = os.path.join(mask_dir, mask_file)
        if os.path.exists(mask_path):
            pairs.append((os.path.join(image_dir, f), mask_path))
        else:
            print("Warning: No mask found for image", f)
    return pairs

train_pairs = get_pairs(train_image_dir, train_mask_dir)
val_pairs = get_pairs(val_image_dir, val_mask_dir)
test_pairs = get_pairs(test_image_dir, test_mask_dir)

print("Total training pairs:", len(train_pairs))
print("Total validation pairs:", len(val_pairs))
print("Total test pairs:", len(test_pairs))


Total training pairs: 369
Total validation pairs: 100
Total test pairs: 232


In [ ]:
# Cell 5: Define the CamVidDataset Class with Mask Remapping

import cv2
import torch
from torch.utils.data import Dataset
import numpy as np

# Mapping dictionary: raw CamVid values to new indices (0 to 14)
mask_mapping = {
    0:   0,
    21:  1,
    33:  2,
    38:  3,
    44:  4,
    56:  5,
    60:  6,
    90:  7,
    97:  8,
    113: 9,
    116: 10,
    120: 11,
    128: 12,
    131: 13,
    184: 14
}

def remap_mask(mask, mapping):
    """
    Remap mask pixel values using the provided mapping.
    """
    remapped = np.zeros_like(mask, dtype=np.int64)
    for orig_val, new_val in mapping.items():
        remapped[mask == orig_val] = new_val
    return remapped

class CamVidDataset(Dataset):
    def __init__(self, pairs, transform=None):
        """
        Args:
            pairs (list): List of (image_path, mask_path) tuples.
            transform (callable, optional): Transformations to apply.
        """
        self.pairs = pairs
        self.transform = transform

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_path, mask_path = self.pairs[idx]

        # Read image and convert to RGB
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Read mask as grayscale and remap it
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = remap_mask(mask, mask_mapping)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented["image"]
            mask = augmented["mask"].long()  # ensure mask is Long
            return image, mask
        else:
            image = cv2.resize(image, (512, 512))
            mask = cv2.resize(mask, (512, 512), interpolation=cv2.INTER_NEAREST)
            image = image.astype(np.float32) / 255.0
            mean = np.array([0.485, 0.456, 0.406], dtype=np.float32)
            std = np.array([0.229, 0.224, 0.225], dtype=np.float32)
            image = (image - mean) / std
            image_tensor = torch.from_numpy(image).permute(2, 0, 1)
            mask_tensor = torch.from_numpy(mask.astype(np.int64))
            return image_tensor, mask_tensor

# (Optional) Verify by checking unique mask values in a few samples
for i in range(5):
    _, mask = CamVidDataset(train_pairs)[i]
    print(f"Sample {i} unique mask values:", np.unique(mask.numpy()))


TypeError: expected np.ndarray (got numpy.ndarray)

In [ ]:
    # Cell 6: Set Up Data Augmentations and Create DataLoaders

    !pip install -q albumentations==1.3.0
    import albumentations as A
    from albumentations.pytorch import ToTensorV2
    from torch.utils.data import DataLoader

    TARGET_SIZE = (512, 512)  # (height, width)

    # Define training transformation with augmentation and normalization
    train_transform = A.Compose([
        A.Resize(*TARGET_SIZE),
        A.HorizontalFlip(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ], additional_targets={"mask": "mask"})

    # Define validation/test transformation (resize and normalization only)
    val_transform = A.Compose([
        A.Resize(*TARGET_SIZE),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ], additional_targets={"mask": "mask"})

    # Create dataset instances with transforms
    train_dataset = CamVidDataset(train_pairs, transform=train_transform)
    val_dataset = CamVidDataset(val_pairs, transform=val_transform)
    test_dataset = CamVidDataset(test_pairs, transform=val_transform)

    # Create DataLoaders for batching
    batch_size = 2  # adjust as needed
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    print("Train samples:", len(train_dataset))
    print("Validation samples:", len(val_dataset))
    print("Test samples:", len(test_dataset))

    import torch
    print("CUDA version:", torch.version.cuda)


In [ ]:
# Cell 7: Set Up DeepLabV3 Model for 15 Classes and Check Model Parameters

import torch.nn as nn
from torchvision import models

num_classes = 15
print("Number of segmentation classes (including background):", num_classes)

# Load the pretrained DeepLabV3 model (default aux_loss=True, 21 classes)
model = models.segmentation.deeplabv3_resnet50(
    weights=models.segmentation.DeepLabV3_ResNet50_Weights.DEFAULT
)

# Replace the main classifier head with a new head for 15 classes (input channels: 2048)
model.classifier = models.segmentation.deeplabv3.DeepLabHead(2048, num_classes)

# Remove the auxiliary classifier by setting it to an identity module
model.aux_classifier = nn.Identity()

# Check model parameters for NaNs or Infs (after model definition)
for name, param in model.named_parameters():
    if torch.isnan(param).any():
        print("Parameter", name, "has NaNs.")
    if torch.isinf(param).any():
        print("Parameter", name, "has Infs.")

# Test moving the model to CPU
try:
    model_cpu = model.cpu()
    print("Model successfully moved to CPU.")
except Exception as e:
    print("Error moving model to CPU:", e)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
try:
    model = model.to(device)
    model.eval()
    print("Model moved to GPU successfully.")
except Exception as e:
    print("Error moving model to GPU:", e)
    print("If you continue to see this error on GPU, consider downgrading PyTorch to a stable version (e.g., 2.0.1) for A100 compatibility.")

# Test a forward pass with a dummy input
with torch.no_grad():
    dummy_in = torch.randn(1, 3, TARGET_SIZE[0], TARGET_SIZE[1]).to(device)
    output = model(dummy_in)
    print("Model output keys:", output.keys())
    print("Main output shape:", output['out'].shape)


In [ ]:
# CELL 8: Define loss functions, optimizer, scheduler, and a 1-epoch training loop with additional metrics and debugging.
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Force synchronous CUDA error reporting

import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# ---------- Define Metric Functions ----------

def compute_confusion_matrix(pred_mask, true_mask, num_classes):
    """
    Computes a confusion matrix for segmentation.
    Both pred_mask and true_mask should be 2D numpy arrays.
    """
    y_pred = pred_mask.flatten()
    y_true = true_mask.flatten()
    cm = np.zeros((num_classes, num_classes), dtype=np.int64)
    for t, p in zip(y_true, y_pred):
        cm[t, p] += 1
    return cm

def compute_iou_from_cm(cm):
    """
    Computes per-class IoU from a confusion matrix.
    Only considers classes where the union is greater than 0.
    """
    IoUs = []
    for i in range(len(cm)):
        TP = cm[i, i]
        FP = cm[:, i].sum() - TP
        FN = cm[i, :].sum() - TP
        union = TP + FP + FN
        if union > 0:
            IoUs.append(TP / union)
    return IoUs

def frequency_weighted_iou(pred_mask, true_mask, num_classes):
    """
    Computes frequency weighted IoU.
    """
    cm = compute_confusion_matrix(pred_mask, true_mask, num_classes)
    freq = np.sum(cm, axis=1) / np.sum(cm)
    ious = compute_iou_from_cm(cm)
    if len(ious) > 0:
        return np.sum(freq[:len(ious)] * np.array(ious))
    else:
        return 0

def dice_coefficient_numpy(pred, target, smooth=1e-6):
    """
    Computes the Dice coefficient (F1 score) between prediction and target.
    Both pred and target should be 2D numpy arrays.
    """
    dice_scores = []
    for cls in range(num_classes):
        pred_cls = (pred == cls).astype(np.float32)
        target_cls = (target == cls).astype(np.float32)
        intersection = np.sum(pred_cls * target_cls)
        union = np.sum(pred_cls) + np.sum(target_cls)
        dice_scores.append((2 * intersection + smooth) / (union + smooth))
    return np.mean(dice_scores)

def compute_segmentation_metrics(pred_mask, true_mask, num_classes):
    """
    Computes segmentation metrics for one sample.
    Returns:
      - per-class IoUs,
      - mean IoU,
      - frequency weighted IoU,
      - Dice coefficient.
    """
    cm = compute_confusion_matrix(pred_mask, true_mask, num_classes)
    ious = compute_iou_from_cm(cm)
    mean_iou = np.mean(ious) if len(ious) > 0 else 0
    fw_iou = frequency_weighted_iou(pred_mask, true_mask, num_classes)
    dice = dice_coefficient_numpy(pred_mask, true_mask)
    return ious, mean_iou, fw_iou, dice

# ---------- Loss Functions ----------

def dice_loss(pred_logits, true_mask, smooth=1e-6):
    num_cls = pred_logits.shape[1]
    # Clamp true_mask values to be within valid range [0, num_cls-1].
    true_mask = true_mask.clamp(0, num_cls - 1)
    true_onehot = F.one_hot(true_mask, num_cls).permute(0, 3, 1, 2).float()
    probas = F.softmax(pred_logits, dim=1)
    dims = (0, 2, 3)
    intersection = torch.sum(probas * true_onehot, dims)
    cardinality = torch.sum(probas + true_onehot, dims)
    dice_per_class = (2. * intersection + smooth) / (cardinality + smooth)
    return 1 - dice_per_class.mean()

def compute_loss(outputs, target):
    """
    Computes total loss as cross-entropy plus Dice loss.
    If an auxiliary output is available, its loss is added with a weight of 0.4.
    """
    if isinstance(outputs, dict):
        out_tensor = outputs['out'].float()
    else:
        out_tensor = outputs.float()

    ce_loss = F.cross_entropy(out_tensor, target)
    d_loss = dice_loss(out_tensor, target)
    loss = ce_loss + d_loss

    if isinstance(outputs, dict) and 'aux' in outputs:
        aux_out = outputs['aux'].float()
        aux_ce_loss = F.cross_entropy(aux_out, target)
        aux_d_loss = dice_loss(aux_out, target)
        loss += 0.4 * (aux_ce_loss + aux_d_loss)
    return loss

# ---------- Optimizer and Scheduler ----------

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
max_epochs = 50  # You can increase to 100 later if needed.
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=max_epochs)

# ---------- Training Loop with Additional Metrics and Ground Truth Debugging ----------

for epoch in range(1, max_epochs + 1):
    model.train()
    epoch_loss = 0.0
    for images, masks in train_loader:
        # Move data to device.
        images = images.to(device)
        masks = masks.to(device)
        # Clamp mask values to ensure they are in [0, num_classes-1].
        masks = masks.clamp(0, num_classes - 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = compute_loss(outputs, masks)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    epoch_loss /= len(train_loader)

    model.eval()
    val_loss = 0.0
    pixel_accs = []
    mean_ious = []
    fw_ious = []
    dice_scores = []

    for images, masks in val_loader:
        images = images.to(device)
        masks = masks.to(device)
        # Clamp masks as well.
        masks = masks.clamp(0, num_classes - 1)
        with torch.no_grad():
            outputs = model(images)
            loss = compute_loss(outputs, masks)
        val_loss += loss.item()

        logits = outputs['out'] if isinstance(outputs, dict) else outputs
        preds = torch.argmax(logits, dim=1)

        # Process the first image in the batch for metrics.
        pred_np = preds[0].cpu().numpy()
        mask_np = masks[0].cpu().numpy()

        # Debug: Print unique ground truth labels for this sample.
        print("Unique ground truth labels for sample:", np.unique(mask_np))

        # Compute pixel accuracy.
        pixel_acc = (pred_np == mask_np).sum() / mask_np.size
        pixel_accs.append(pixel_acc)

        # Compute segmentation metrics.
        _, mean_iou, fw_iou, dice_val = compute_segmentation_metrics(pred_np, mask_np, num_classes)
        mean_ious.append(mean_iou)
        fw_ious.append(fw_iou)
        dice_scores.append(dice_val)

    val_loss /= len(val_loader)
    avg_pixel_acc = np.mean(pixel_accs)
    avg_mean_iou = np.mean(mean_ious)
    avg_fw_iou = np.mean(fw_ious)
    avg_dice = np.mean(dice_scores)

    # Update the learning rate scheduler.
    scheduler.step()

    print(f"Epoch {epoch:02d}: Train Loss = {epoch_loss:.4f}, Val Loss = {val_loss:.4f}, "
          f"Val Pixel Acc = {avg_pixel_acc:.4f}, Val Mean IoU = {avg_mean_iou:.4f}, "
          f"Val Freq Weighted IoU = {avg_fw_iou:.4f}, Val Dice/F1 = {avg_dice:.4f}")


In [ ]:
# Cell 9: Visualize Predictions on the Validation Set

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import ListedColormap

# Create a custom colormap for visualization (for 15 classes)
colors_15 = [
    "black", "red", "green", "blue", "orange", "purple", "cyan", "magenta",
    "yellow", "brown", "pink", "gray", "lime", "lavender", "turquoise"
]
custom_cmap = ListedColormap(colors_15[:num_classes])

def mask_to_color(mask, colormap):
    """
    Converts a 2D mask (tensor or array) to an RGB image using the provided colormap.
    """
    mask_np = mask.cpu().numpy() if isinstance(mask, torch.Tensor) else mask
    h, w = mask_np.shape
    color_mask = np.zeros((h, w, 3), dtype=np.uint8)
    for cls_idx, color in enumerate(colormap.colors):
        rgb_color = (np.array(mcolors.to_rgb(color)) * 255).astype(np.uint8)
        color_mask[mask_np == cls_idx] = rgb_color
    return color_mask

model.eval()
# Visualize the first 5 samples from the validation set.
for i in range(5):
    image, mask = val_dataset[i]
    # Reverse the normalization for display.
    img_np = image.permute(1, 2, 0).cpu().numpy()
    mean_vis = np.array([0.485, 0.456, 0.406])
    std_vis = np.array([0.229, 0.224, 0.225])
    img_np = img_np * std_vis + mean_vis
    img_np = (img_np * 255).astype(np.uint8)

    # Generate prediction.
    with torch.no_grad():
        image_input = image.unsqueeze(0).to(device)
        output = model(image_input)
        pred_mask = torch.argmax(output['out'], dim=1)[0]

    gt_mask_color = mask_to_color(mask, custom_cmap)
    pred_mask_color = mask_to_color(pred_mask, custom_cmap)

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.imshow(img_np)
    plt.title("Input Image")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(gt_mask_color)
    plt.title("Ground Truth Mask")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(pred_mask_color)
    plt.title("Predicted Mask")
    plt.axis('off')
    plt.show()

print("Unique predicted classes in last sample:", torch.unique(pred_mask))


In [ ]:
# Cell 10: Compute per-class metrics on the test set

import numpy as np
import pandas as pd

# derive raw mask pixel‐value for each class‐index
inv_map = {v:k for k,v in mask_mapping.items()}
class_raw_vals = [inv_map[i] for i in range(num_classes)]
# make placeholder names – replace these with your real class names if you have them
class_names = [f"label{v}" for v in class_raw_vals]

# initialize aggregate confusion matrix
confusion = np.zeros((num_classes, num_classes), dtype=np.int64)

model.eval()
with torch.no_grad():
    for images, masks in test_loader:
        images = images.to(device)
        masks  = masks.to(device)
        outputs = model(images)
        preds   = torch.argmax(outputs['out'], dim=1)

        pred_np = preds[0].cpu().numpy()
        mask_np = masks[0].cpu().numpy()
        confusion += compute_confusion_matrix(pred_np, mask_np, num_classes)

# now compute per-class PixelAcc, IoU and F1
rows = []
for i, name in enumerate(class_names):
    TP = confusion[i, i]
    FP = confusion[:, i].sum() - TP
    FN = confusion[i, :].sum() - TP
    union = TP + FP + FN

    pix_acc = TP / confusion[i, :].sum() if confusion[i, :].sum() > 0 else np.nan
    iou     = TP / union               if union               > 0 else np.nan
    f1      = 2*TP / (2*TP + FP + FN)  if (2*TP + FP + FN)  > 0 else np.nan

    rows.append((name, pix_acc, iou, f1))

df = pd.DataFrame(rows, columns=["Class","PixelAcc","IoU","F1"])
print(df)
